In [1]:
from tqdm import tqdm
from models import CompletionNetwork, ContextDiscriminator, GlobalDiscriminator_P
from datasets_inference import ImageDataset
from losses import completion_network_loss, completion_network_loss_P
from noise import AddGaussianNoise
from utils import (
    gen_input_mask,
    gen_hole_area,
    crop,
    sample_random_batch,
    poisson_blend,
)
from torch.utils.data import DataLoader
from torch.optim import Adadelta, Adam
from torch.nn import BCELoss, DataParallel
from torchvision.utils import save_image
import torchvision.utils as vutils
from PIL import Image
import torchvision.transforms as transforms
import torch
import random
import os
import argparse
import numpy as np
import json
import time

import cv2
import matplotlib.pyplot as plt
import glob

In [2]:
!pwd

/root/notebooks/final/pedestrian_generator


In [3]:
# data_path = "/root/notebooks/final/caltech_origin_mask2_20000"
# out_path = "/root/notebooks/final/caltech_origin_mask_20000/result_6-10_part2"

data_path = "/root/notebooks/final/caltech_origin_mask2_20000"

model_path = "/root/notebooks/final/caltech_origin_mask_20000/result_6-10_part2/phase_3/1_model_cn_step4800"

out_path = "/root/notebooks/final/caltech_origin_mask2_20000"

In [7]:
sorted(glob.glob(data_path+'/people/*'), key=lambda x: x[-10:-3])[:10]

['/root/notebooks/final/caltech_origin_mask2_20000/people/000000.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000001.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000002.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000003.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000004.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000005.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000006.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000007.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000008.jpg',
 '/root/notebooks/final/caltech_origin_mask2_20000/people/000009.jpg']

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default=data_path)
parser.add_argument('--result_dir', type=str, default=out_path)
parser.add_argument('--recursive_search', action='store_true', default=False)
parser.add_argument('--init_model_cn', type=str, default=model_path)
parser.add_argument('--init_model_cd', type=str, default=None)
parser.add_argument('--steps_1', type=int, default=8000)
parser.add_argument('--steps_2', type=int, default=4000)
parser.add_argument('--steps_3', type=int, default=3000)
parser.add_argument('--snaperiod_1', type=int, default=800)
parser.add_argument('--snaperiod_2', type=int, default=400)
parser.add_argument('--snaperiod_3', type=int, default=300)
parser.add_argument('--max_holes', type=int, default=1)
parser.add_argument('--hole_min_w', type=int, default=48)
parser.add_argument('--hole_max_w', type=int, default=96)
parser.add_argument('--hole_min_h', type=int, default=48)
parser.add_argument('--hole_max_h', type=int, default=96)
parser.add_argument('--cn_input_size', type=int, default=256)
parser.add_argument('--ld_input_size', type=int, default=256)
parser.add_argument('--optimizer', type=str, choices=['adadelta', 'adam'], default='adam')
parser.add_argument('--bsize', type=int, default=24)
parser.add_argument('--bdivs', type=int, default=1)
parser.add_argument('--data_parallel', action='store_true')
parser.add_argument('--num_test_completions', type=int, default=3)
parser.add_argument('--mpv', nargs=3, type=float, default=None)
parser.add_argument('--alpha', type=float, default=4e-3)
parser.add_argument('--arc', type=str, choices=['celeba', 'places2'], default='celeba')

parser.add_argument('-f')

_StoreAction(option_strings=['-f'], dest='f', nargs=None, const=None, default=None, type=None, choices=None, help=None, metavar=None)

In [9]:
args = parser.parse_args()

In [10]:
# ================================================
# Preparation
# ================================================
args.data_dir = os.path.expanduser(args.data_dir)
args.result_dir = os.path.expanduser(args.result_dir)
if args.init_model_cn != None:
    args.init_model_cn = os.path.expanduser(args.init_model_cn)


if torch.cuda.is_available() == False:
    raise Exception('At least one gpu must be available.')
else:
    gpu = torch.device('cuda:0')

# create result directory (if necessary)
if os.path.exists(args.result_dir) == False:
    os.makedirs(args.result_dir)

    
#code below not used    
for s in ['output']:
    if os.path.exists(os.path.join(args.result_dir, s)) == False:
        os.makedirs(os.path.join(args.result_dir, s))


In [11]:
# imgs = np.einsum('klij->kjli', np.array(imgs))
# print(imgs.shape)

In [12]:
# inference dataset

trnsfm = transforms.Compose([
    transforms.ToTensor(),
    
])

trnsfm2 = transforms.Compose([
    transforms.ToTensor(),
])
print('loading dataset... (it may take a few minutes)')
train_dset = ImageDataset(args.data_dir, trnsfm,trnsfm2, load2meme = False)
train_loader = DataLoader(train_dset, batch_size=(args.bsize // args.bdivs), shuffle=False)

alpha = torch.tensor(args.alpha).to(gpu)

loading dataset... (it may take a few minutes)


In [13]:
# Create model G

model_cn = CompletionNetwork()
if args.data_parallel:
    model_cn = DataParallel(model_cn)
    
if args.init_model_cn != None:
    model_cn.load_state_dict(torch.load(args.init_model_cn, map_location='cpu'))

    
model_cn = model_cn.to(gpu)


In [22]:
for street_img, mask_poeple, mask, centers, img_names in train_loader:
        
    street_img = street_img.to(gpu)
    mask_poeple = mask_poeple.to(gpu)
    mask = mask

    input = torch.cat((street_img, mask_poeple), dim=1)
    with torch.no_grad():
        output = model_cn(input)

    mask_ = (mask).le(0.5).to(torch.uint8) * 255

    completed = poisson_blend(output, street_img, mask_)
    break

In [26]:
street_img_pil = Image.open(img_names[10])
mask_pil = transPIL(mask[10]).convert("RGBA")
completed_pil = transPIL(completed.cpu()[10]).convert("RGBA")

street_img_pil.paste(completed_pil,(centers[10][0]-128,centers[10][1]-128),mask_pil)

imgpath = str(img_names[10])
# imgpath = imgpath.replace('street', 'output')
# #print(imgpath)
# save_image(img, '/root/notebooks/final/caltech_origin_mask2_20000/test.jpg')
# street_img_pil.save('')

street_img_pil.save('/root/notebooks/final/caltech_origin_mask2_20000/test.jpg')



In [24]:
imgpath

'/root/notebooks/final/caltech_origin_mask2_20000/street/000010.jpg'

In [25]:
transPIL = transforms.ToPILImage()

In [19]:
def inference_G():
    cnt_bdivs = 0
    pbar = tqdm(total=len(train_loader))

    for street_img, mask_poeple, mask, centers, img_names in train_loader:
        
        street_img = street_img.to(gpu)
        mask_poeple = mask_poeple.to(gpu)
        mask = mask

        input = torch.cat((street_img, mask_poeple), dim=1)
        with torch.no_grad():
            output = model_cn(input)
            
        mask_ = (mask).le(0.5).to(torch.uint8) * 255
        
        completed = poisson_blend(output, street_img, mask_)
        
        for i in range(len(completed)):
            img = completed[i]
            
            street_img_pil = Image.open(img_names[i])
            mask_pil = transPIL(mask[i]).convert("RGBA")
            completed_pil = transPIL(completed.cpu()[i]).convert("RGB")
            
            street_img_pil.paste(completed_pil,(centers[i][0]-128,centers[i][1]-128),mask_pil.convert("L"))
            
            imgpath = str(img_names[i])
            imgpath = imgpath.replace('street', 'output')
            #print(imgpath)
            #save_image(img, imgpath)
            street_img_pil.save(imgpath)
        pbar.update()
            
        #pbar.set_description('%d | phase 1 | train loss: %.5f' % (n,loss.cpu()))
        
    pbar.close()


In [20]:
inference_G()




 29%|██▉       | 120/417 [06:15<15:23,  3.11s/it]


 29%|██▉       | 121/417 [06:18<15:23,  3.12s/it]


 29%|██▉       | 122/417 [06:21<15:18,  3.11s/it]


 29%|██▉       | 123/417 [06:24<15:14,  3.11s/it]


 30%|██▉       | 124/417 [06:28<15:10,  3.11s/it]


 30%|██▉       | 125/417 [06:31<15:12,  3.13s/it]


 30%|███       | 126/417 [06:34<15:12,  3.14s/it]


 30%|███       | 127/417 [06:37<15:00,  3.11s/it]


 31%|███       | 128/417 [06:40<15:15,  3.17s/it]


 31%|███       | 129/417 [06:43<15:16,  3.18s/it]


 31%|███       | 130/417 [06:47<15:13,  3.18s/it]


 31%|███▏      | 131/417 [06:50<15:06,  3.17s/it]


 32%|███▏      | 132/417 [06:53<14:53,  3.14s/it]


 32%|███▏      | 133/417 [06:56<14:46,  3.12s/it]


 32%|███▏      | 134/417 [06:59<14:34,  3.09s/it]


 32%|███▏      | 135/417 [07:02<14:29,  3.08s/it]


 33%|███▎      | 136/417 [07:05<14:30,  3.10s/it]


 33%|███▎      | 137/417 [07:08<14:43,  3.16s/it]


 33%|███▎      | 138/417 [07:11<14:31,  3.12s/it]


 33%|███▎